# ROM 102 - Practical Considerations

In ROM 101, we learned about reduced order models (**ROM**s), proper orthogonal decomposition (**POD**), and **Galerkin** projection, and we saw the results of some applications of **POD-Galerkin ROM**s.  

In this lecture, we'll 

  1. describe transient heat conduction in a nuclear reactor fuel element
  2. construct a full-order model (**FOM**) for this application using a finite-difference (**FD**) approximation 
  3. construct a corresponding POD-Galerkin ROM
  4. analyze 

# Application: Transient Conduction

The conduction of heat in solids can be described by

$$
  \rho(\mathbf{r})c_p(\mathbf{r}, T) \frac{\partial T}{\partial t} 
    = \nabla k(\mathbf{r}, T) \nabla T(\mathbf{r}, t) + q'''(\mathbf{r}, t) \, .
    \tag{1}
$$

where $T$ is temperature, $\rho$ is density, $c_p$ is specific heat, $k$ is thermal conductivity, and $q'''$ is the volumetric heat generation rate.

As an example application, consider the conduction of heat in a nuclear reactor fuel element.  A fuel element is typically a long cylinder ($L=3$ m; $D = 0.9$cm) made of ceramic UO$_2$ fuel enclosed in a thin ($\Delta = 0.1$ cm) metallic sheath called the cladding.  Often, there is a small gap between the fuel and the cladding filled with an inert gas like argon.  Reactions from fission in the fuel leads to local heating, and, given the volumetric fission rate $f(\mathbf{r}, t)$, the heat generation rate is $q''' = \kappa f$ where $\kappa$ is a conversion factor. These fuel elements are most often assembled in relatively tight ($1.2$ cm center-to-center), vertical clusters cooled by forced flow of water through the channels formed by the exterior of the cladding.

For simplicity, we'll consider thin plate fuel elements in which conduction occurs only along the $x$ axis, or

$$
  \rho(x)c_p(x, T) \frac{\partial T}{\partial t} 
    = \frac{\partial}{\partial x} \left ( k(x, T) \frac{\partial}{\partial x}  T(x, t)\right ) + q'''(x, t) \, .
    \tag{2}
$$

In particular, we'll consider a geometry that looks like

```
    |           |        |
    |   UO2     |  CLAD  |   COOLANT
    |           |        |
   x=0         x=x_f    x=x_c
```

and set 

$$
   q''(0) = 0
   \tag{3}
$$

and 

$$
  q''(x_c) = h(T(x_c)-T_{cool}) \, .
  \tag{4}
$$ 

In other words, we're treating a single plate element with a fixed, convective condition at the coolant boundary.

# Full-Order Model via Finite Differences

To solve (2), we'll use the finite-difference method.  Let the spatial domain be discretized s

In [40]:
def k_fuel(T_K, B=0.0) : 
    """ Calculate fuel conductivity (W/m-K) for temperature T_K (K)
    
    From J.D. Hales et al. (2013) "Bison Theory" (NFIR model)
    """
    from numpy import exp, tanh
    # kelvin to celsius
    T = T_K - 273.15;
    # thermal recovery function
    rf = 0.5*(1.0+tanh((T-900.0)/150.0))
    # phonon contribution at start of thermal recovery [Hales eq. 8.14]
    kps = 1.0/(0.09592+0.00614*B-0.000014*B**2+(0.00025-0.00000181*B)*T)
    # phonon contribution at the end of thermal recovery [Hales eq. 8.15]
    kpend = 1.0/(0.09592+0.0026*B+(0.00025-0.00000027*B)*T)
    # unirradiated material at 95% th. density [Hales eq. 8.17]
    kel = 0.0132*exp((0.00188)*T)
    k = (1.0-rf)*kps + rf*kpend + kel
    return k

def k_cladding(T) :
    """ Returns thermal conductivity (W/m) for Zr-4 at temperature T (K)
    
    Reference: INL/EXT-14-32591, Revision 1
    """
    return 7.511 + 2.088e-2*T - 1.45e-5*T**2 + 7.668e-9*T**3

In [42]:
k_cladding(600)

16.475288